In [1]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
import os
import json 
from pandas.io.json import json_normalize

In [2]:
raw_df_women = pd.read_csv("rawdata/womens_shoe_prices.csv", error_bad_lines=False, index_col=None)
raw_df_men = pd.read_csv("rawdata/mens_shoe_prices.csv", error_bad_lines=False, index_col=None)
raw_df = raw_df_women.append(raw_df_men)
raw_df=raw_df.reset_index(drop=True)
raw_df

/anaconda3/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (25,36,39,44) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
b'Skipping line 251: expected 48 fields, saw 49\nSkipping line 444: expected 48 fields, saw 50\nSkipping line 847: expected 48 fields, saw 49\nSkipping line 848: expected 48 fields, saw 49\nSkipping line 1018: expected 48 fields, saw 49\nSkipping line 1575: expected 48 fields, saw 51\nSkipping line 2133: expected 48 fields, saw 49\nSkipping line 2922: expected 48 fields, saw 51\nSkipping line 3777: expected 48 fields, saw 51\nSkipping line 4057: expected 48 fields, saw 49\nSkipping line 4239: expected 48 fields, saw 49\nSkipping line 4240: expected 48 fields, saw 49\nSkipping line 4384: expected 48 fields, saw 49\nSkipping line 4385: expected 48 fields, saw 49\nSkipping line 5388: expected 48 fields, saw 49\nSkipping line

,asins,brand,categories,colors,count,dateAdded,dateUpdated,descriptions,dimension,ean,...,prices.warranty,quantities,reviews,sizes,skus,sourceURLs,upc,vin,websiteIDs,weight
0,NaN,Zoot,"Shoes,Clothing,Women's Shoes,All Women's Shoes","Blue,Multicolor",NaN,2016-11-11T09:49:00Z,2016-11-11T09:49:00Z,"[{""dateSeen"":[""2016-11-11T09:49:00Z""],""sourceU...",NaN,NaN,...,NaN,NaN,NaN,"10,9",NaN,https://www.walmart.com/ip/Zoot-TT-TRAINER-2.0...,NaN,NaN,NaN,NaN
1,NaN,Zoot,"Shoes,Clothing,Women's Shoes,All Women's Shoes","Blue,Multicolor",NaN,2016-11-11T09:49:00Z,2016-11-11T09:49:00Z,"[{""dateSeen"":[""2016-11-11T09:49:00Z""],""sourceU...",NaN,NaN,...,NaN,NaN,NaN,"10,9",NaN,https://www.walmart.com/ip/Zoot-TT-TRAINER-2.0...,NaN,NaN,NaN,NaN
2,NaN,Zoot,"Shoes,Clothing,Women's Shoes,All Women's Shoes","Blue,Multicolor",NaN,2016-11-11T09:49:00Z,2016-11-11T09:49:00Z,"[{""dateSeen"":[""2016-11-11T09:49:00Z""],""sourceU...",NaN,NaN,...,NaN,NaN,NaN,"10,9",NaN,https://www.walmart.com/ip/Zoot-TT-TRAINER-2.0...,NaN,NaN,NaN,NaN
3,NaN,Zoot,"Shoes,Clothing,Women's Shoes,All Women's Shoes","Blue,Multicolor",NaN,2016-11-11T09:49:00Z,2016-11-11T09:49:00Z,"[{""dateSeen"":[""2016-11-11T09:49:00Z""],""sourceU...",NaN,NaN,...,NaN,NaN,NaN,"10,9",NaN,https://www.walmart.com/ip/Zoot-TT-TRAINER-2.0...,NaN,NaN,NaN,NaN
4,NaN,Wild Pair,"Shoes,Women's Shoes,Clothing,All Women's Shoes",Brown,NaN,2016-11-16T12:56:36Z,2016-11-16T12:56:36Z,"[{""dateSeen"":[""2016-11-16T12:56:36Z""],""sourceU...",NaN,7.631810e+11,...,NaN,NaN,NaN,"8.5,7,7.5,8",NaN,https://www.walmart.com/ip/Wild-Pair-Colfax-Wo...,7.63181E+11,NaN,NaN,NaN
5,NaN,Wild Pair,"Shoes,Women's Shoes,Clothing,All Women's Shoes",Brown,NaN,2016-11-16T12:56:36Z,2016-11-16T12:56:36Z,"[{""dateSeen"":[""2016-11-16T12:56:36Z""],""sourceU...",NaN,7.631810e+11,...,NaN,NaN,NaN,"8.5,7,7.5,8",NaN,https://www.walmart.com/ip/Wild-Pair-Colfax-Wo...,7.63181E+11,NaN,NaN,NaN
6,NaN,Wild Pair,"Shoes,Women's Shoes,Clothing,All Women's Shoes",Brown,NaN,2016-11-16T12:56:36Z,2016-11-16T12:56:36Z,"[{""dateSeen"":[""2016-11-16T12:56:36Z""],""sourceU...",NaN,7.631810e+11,...,NaN,NaN,NaN,"8.5,7,7.5,8",NaN,https://www.walmart.com/ip/Wild-Pair-Colfax-Wo...,7.63181E+11,NaN,NaN,NaN
7,NaN,Sutton Studio,"Women's Suits & Sets,Women's Clothing,All Wome...",Pink,NaN,2016-01-02T03:16:24Z,2016-04-06T03:01:11Z,NaN,NaN,NaN,...,NaN,NaN,NaN,8,"[{""sourceURLs"":[""http://www.sears.com/content/...",http://www.sears.com/content/pdp/products/pric...,NaN,NaN,NaN,NaN
8,NaN,Sutton Studio,"Women's Suits & Sets,Women's Clothing,All Wome...",Pink,NaN,2016-01-02T03:16:24Z,2016-04-06T03:01:11Z,NaN,NaN,NaN,...,NaN,NaN,NaN,8,"[{""sourceURLs"":[""http://www.sears.com/content/...",http://www.sears.com/content/pdp/products/pric...,NaN,NaN,NaN,NaN
9,NaN,LaLeela.com,"Women's Clothing,All Women's Clothing,Clothing...",NaN,NaN,2016-03-11T09:11:01Z,2016-03-25T01:13:39Z,NaN,1 in x 1 in x 1 in,NaN,...,NaN,NaN,NaN,NaN,"[{""sourceURLs"":[""http://www.sears.com/content/...",http://www.sears.com/content/pdp/products/pric...,NaN,NaN,NaN,NaN


In [3]:
df = raw_df[["features","name","prices.amountMin","prices.isSale"]]
df.head()

,features,name,prices.amountMin,prices.isSale
0,"[{""key"":""Season"",""value"":[""All-Season""]},{""key...",Zoot Tt Trainer 2.0 Round Toe Synthetic Sne...,71.99,TRUE
1,"[{""key"":""Season"",""value"":[""All-Season""]},{""key...",Zoot Tt Trainer 2.0 Round Toe Synthetic Sne...,250,FALSE
2,"[{""key"":""Season"",""value"":[""All-Season""]},{""key...",Zoot Tt Trainer 2.0 Round Toe Synthetic Sne...,75.99,TRUE
3,"[{""key"":""Season"",""value"":[""All-Season""]},{""key...",Zoot Tt Trainer 2.0 Round Toe Synthetic Sne...,79.98,TRUE
4,"[{""key"":""Heel Height"",""value"":[""High (3 in. an...",Wild Pair Colfax Women Peep-toe Synthetic Bro...,26.98,TRUE


In [4]:
# replace true and false with TRUE and FALSE
df["prices.isSale"] = df["prices.isSale"].map({"true":"TRUE", "false":"FALSE", "TRUE":"TRUE", "FALSE":"FALSE"})
df.head()

/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,features,name,prices.amountMin,prices.isSale
0,"[{""key"":""Season"",""value"":[""All-Season""]},{""key...",Zoot Tt Trainer 2.0 Round Toe Synthetic Sne...,71.99,TRUE
1,"[{""key"":""Season"",""value"":[""All-Season""]},{""key...",Zoot Tt Trainer 2.0 Round Toe Synthetic Sne...,250,FALSE
2,"[{""key"":""Season"",""value"":[""All-Season""]},{""key...",Zoot Tt Trainer 2.0 Round Toe Synthetic Sne...,75.99,TRUE
3,"[{""key"":""Season"",""value"":[""All-Season""]},{""key...",Zoot Tt Trainer 2.0 Round Toe Synthetic Sne...,79.98,TRUE
4,"[{""key"":""Heel Height"",""value"":[""High (3 in. an...",Wild Pair Colfax Women Peep-toe Synthetic Bro...,26.98,TRUE


In [5]:
len(df)

38360

In [6]:
df = df.dropna()

In [7]:
len(df)

26065

In [8]:
df.loc[0,"name"]

'Zoot Tt Trainer 2.0   Round Toe Synthetic  Sneakers'

In [9]:
# remove non-shoe rows:
outlier_list = ['pants','jeans', 'jacket','cardigan', 'dress','shirt','romper','suit','pedicure', 'bra','underwire','wirefree','bikini','swim','lingerie','thong','slippers','pantyhose','hairpin',
              'swimsuit','swimwear','scarf','glove','leggings','eyewear','Sunglasses',"Costume",
              'dress','blazer', 'skirt','coat','top','socks','Doll','sleepwear','makeup',
              'bracelet','necklace','jewelry','diamond','band','ring','watch','Parka','lens']

for outlier in outlier_list:
    df = df[df.name.str.contains(outlier) == False]
df

,features,name,prices.amountMin,prices.isSale
0,"[{""key"":""Season"",""value"":[""All-Season""]},{""key...",Zoot Tt Trainer 2.0 Round Toe Synthetic Sne...,71.99,TRUE
1,"[{""key"":""Season"",""value"":[""All-Season""]},{""key...",Zoot Tt Trainer 2.0 Round Toe Synthetic Sne...,250,FALSE
2,"[{""key"":""Season"",""value"":[""All-Season""]},{""key...",Zoot Tt Trainer 2.0 Round Toe Synthetic Sne...,75.99,TRUE
3,"[{""key"":""Season"",""value"":[""All-Season""]},{""key...",Zoot Tt Trainer 2.0 Round Toe Synthetic Sne...,79.98,TRUE
4,"[{""key"":""Heel Height"",""value"":[""High (3 in. an...",Wild Pair Colfax Women Peep-toe Synthetic Bro...,26.98,TRUE
5,"[{""key"":""Heel Height"",""value"":[""High (3 in. an...",Wild Pair Colfax Women Peep-toe Synthetic Bro...,15.99,TRUE
6,"[{""key"":""Heel Height"",""value"":[""High (3 in. an...",Wild Pair Colfax Women Peep-toe Synthetic Bro...,90,FALSE
7,"[{""key"":""Style"",""value"":[""Suit jackets""]},{""ke...",Sutton Studio Women's 100 Cashmere Blazer Jacket,318,FALSE
8,"[{""key"":""Style"",""value"":[""Suit jackets""]},{""ke...",Sutton Studio Women's 100 Cashmere Blazer Jacket,166.99,TRUE
9,"[{""key"":""Color"",""value"":[""Purple""]}]",Laleela.com Women Supersoft Cotton Coverup Bat...,27.98,TRUE


In [10]:
len(df)

23727

In [11]:
# convert string to float
df["prices.amountMin"] = pd.to_numeric(df["prices.amountMin"], errors='coerce')

In [12]:
# only keep rows with min and max price for each item
df_min = df.sort_values("prices.amountMin").groupby("name", as_index=False).first()
df_max = df.sort_values("prices.amountMin", ascending=False).groupby("name", as_index=False).first()
df = df_min.append(df_max)
df = df.reset_index(drop=True)
df = df.drop_duplicates()

In [13]:
len(df)

18787

In [14]:
df.sort_values("name")

,name,features,prices.amountMin,prices.isSale
0,$125 Brand New - Nike Tiger Woods Tw Adaptive ...,"[{""key"":""Sport"",""value"":[""Leisure, Golf""]},{""k...",65.00,FALSE
1,$125 Dansko Mens Professional Box Leather Clog...,"[{""key"":""Style"",""value"":[""Nursing & Uniform""]}...",58.31,FALSE
2,$125 Dansko Professional Leather Clogs In Hick...,"[{""key"":""Heel Height"",""value"":[""Med (1 3/4 in....",55.25,TRUE
3,$125 Lauren Ralph Lauren Fully Lined Golf - Dr...,"[{""key"":""Style"",""value"":[""Golf - Dress - Pleat...",49.99,FALSE
4,$125 Lauren Ralph Lauren Lined Wool/poly Dress...,"[{""key"":""Style"",""value"":[""Dress - Pleat""]},{""k...",59.99,FALSE
5,$125 New Nike Women's Sz 6.5 Dunk Sky Hi Casua...,"[{""key"":""Style"",""value"":[""Fashion Sneakers""]},...",73.95,FALSE
12376,$125 New Nike Women's Sz 6.5 Dunk Sky Hi Casua...,"[{""key"":""Style"",""value"":[""Fashion Sneakers""]},...",87.95,FALSE
6,$125 Nwt Nautica Gray Flat Front Dress Pants T...,"[{""key"":""Style"",""value"":[""Dress - Flat Front""]...",19.99,FALSE
7,$125 Nwt Nautica Travel Wool Blend Blue Checke...,"[{""key"":""Style"",""value"":[""Dress - Pleat""]},{""k...",24.99,FALSE
8,$125 Nwt Nautica Travelwear Wool Blend Black P...,"[{""key"":""Style"",""value"":[""Dress - Pleat""]},{""k...",19.99,FALSE


In [15]:
df.loc[0,"features"]

'[{"key":"Sport","value":["Leisure, Golf"]},{"key":"Style","value":["Pants"]},{"key":"Condition","value":["New with tags"]}]'

In [16]:
# convert string to list
from ast import literal_eval
a = literal_eval(df.loc[0,"features"])
a

[{'key': 'Sport', 'value': ['Leisure, Golf']},
 {'key': 'Style', 'value': ['Pants']},
 {'key': 'Condition', 'value': ['New with tags']}]

In [17]:
for i in range(len(df)):
    try:
        df.at[i,"features"] = literal_eval(df.loc[i,"features"])
    except:
        pass

In [18]:
df.head()

,name,features,prices.amountMin,prices.isSale
0,$125 Brand New - Nike Tiger Woods Tw Adaptive ...,"[{'key': 'Sport', 'value': ['Leisure, Golf']},...",65.00,FALSE
1,$125 Dansko Mens Professional Box Leather Clog...,"[{'key': 'Style', 'value': ['Nursing & Uniform...",58.31,FALSE
2,$125 Dansko Professional Leather Clogs In Hick...,"[{'key': 'Heel Height', 'value': ['Med (1 3/4 ...",55.25,TRUE
3,$125 Lauren Ralph Lauren Fully Lined Golf - Dr...,"[{'key': 'Style', 'value': ['Golf - Dress - Pl...",49.99,FALSE
4,$125 Lauren Ralph Lauren Lined Wool/poly Dress...,"[{'key': 'Style', 'value': ['Dress - Pleat']},...",59.99,FALSE


In [19]:
df["features"].apply(pd.Series)

,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
0,"{'key': 'Sport', 'value': ['Leisure, Golf']}","{'key': 'Style', 'value': ['Pants']}","{'key': 'Condition', 'value': ['New with tags']}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'key': 'Style', 'value': ['Nursing & Uniform']}","{'key': 'Material', 'value': ['Leather']}","{'key': 'Condition', 'value': ['Pre-owned']}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'key': 'Heel Height', 'value': ['Med (1 3/4 i...","{'key': 'Pattern', 'value': ['Solid']}","{'key': 'Occasion', 'value': ['Wear to Work']}","{'key': 'Material', 'value': ['Leather']}","{'key': 'Country/Region of Manufacture', 'valu...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'key': 'Style', 'value': ['Golf - Dress - Ple...","{'key': 'Inseam', 'value': ['30']}","{'key': 'Material', 'value': ['65% Polyester -...","{'key': 'Front Style', 'value': ['Pleated fron...","{'key': 'Country/Region of Manufacture', 'valu...","{'key': 'Condition', 'value': ['New with tags']}",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'key': 'Style', 'value': ['Dress - Pleat']}","{'key': 'Material', 'value': ['50% Wool - 50% ...","{'key': 'Condition', 'value': ['New with tags']}","{'key': 'Country of Manufacture', 'value': ['I...","{'key': 'Color', 'value': ['Gray']}",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"{'key': 'Style', 'value': ['Fashion Sneakers']}","{'key': 'Condition', 'value': ['New with box']}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"{'key': 'Style', 'value': ['Dress - Flat Front']}","{'key': 'Inseam', 'value': ['30']}","{'key': 'Country/Region of Manufacture', 'valu...","{'key': 'Condition', 'value': ['New with tags']}",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"{'key': 'Style', 'value': ['Dress - Pleat']}","{'key': 'Inseam', 'value': ['30']}","{'key': 'Country/Region of Manufacture', 'valu...","{'key': 'Condition', 'value': ['New with tags']}",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"{'key': 'Style', 'value': ['Dress - Pleat']}","{'key': 'Inseam', 'value': ['32']}","{'key': 'Country/Region of Manufacture', 'valu...","{'key': 'Condition', 'value': ['New with tags']}",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"{'key': 'Style', 'value': ['Vest']}","{'key': 'Pattern', 'value': ['Solid']}","{'key': 'Condition', 'value': ['New with tags']}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
type(df["features"].apply(pd.Series))

pandas.core.frame.DataFrame

In [21]:
new_df = pd.concat([df["features"].apply(pd.Series), df["name"], df["prices.amountMin"], df["prices.isSale"]], axis = 1)
new_df

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,name,prices.amountMin,prices.isSale
0,"{'key': 'Sport', 'value': ['Leisure, Golf']}","{'key': 'Style', 'value': ['Pants']}","{'key': 'Condition', 'value': ['New with tags']}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,$125 Brand New - Nike Tiger Woods Tw Adaptive ...,65.00,FALSE
1,"{'key': 'Style', 'value': ['Nursing & Uniform']}","{'key': 'Material', 'value': ['Leather']}","{'key': 'Condition', 'value': ['Pre-owned']}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,$125 Dansko Mens Professional Box Leather Clog...,58.31,FALSE
2,"{'key': 'Heel Height', 'value': ['Med (1 3/4 i...","{'key': 'Pattern', 'value': ['Solid']}","{'key': 'Occasion', 'value': ['Wear to Work']}","{'key': 'Material', 'value': ['Leather']}","{'key': 'Country/Region of Manufacture', 'valu...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,$125 Dansko Professional Leather Clogs In Hick...,55.25,TRUE
3,"{'key': 'Style', 'value': ['Golf - Dress - Ple...","{'key': 'Inseam', 'value': ['30']}","{'key': 'Material', 'value': ['65% Polyester -...","{'key': 'Front Style', 'value': ['Pleated fron...","{'key': 'Country/Region of Manufacture', 'valu...","{'key': 'Condition', 'value': ['New with tags']}",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,$125 Lauren Ralph Lauren Fully Lined Golf - Dr...,49.99,FALSE
4,"{'key': 'Style', 'value': ['Dress - Pleat']}","{'key': 'Material', 'value': ['50% Wool - 50% ...","{'key': 'Condition', 'value': ['New with tags']}","{'key': 'Country of Manufacture', 'value': ['I...","{'key': 'Color', 'value': ['Gray']}",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,$125 Lauren Ralph Lauren Lined Wool/poly Dress...,59.99,FALSE
5,"{'key': 'Style', 'value': ['Fashion Sneakers']}","{'key': 'Condition', 'value': ['New with box']}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,$125 New Nike Women's Sz 6.5 Dunk Sky Hi Casua...,73.95,FALSE
6,"{'key': 'Style', 'value': ['Dress - Flat Front']}","{'key': 'Inseam', 'value': ['30']}","{'key': 'Country/Region of Manufacture', 'valu...","{'key': 'Condition', 'value': ['New with tags']}",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,$125 Nwt Nautica Gray Flat Front Dress Pants T...,19.99,FALSE
7,"{'key': 'Style', 'value': ['Dress - Pleat']}","{'key': 'Inseam', 'value': ['30']}","{'key': 'Country/Region of Manufacture', 'valu...","{'key': 'Condition', 'value': ['New with tags']}",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,$125 Nwt Nautica Travel Wool Blend Blue Checke...,24.99,FALSE
8,"{'key': 'Style', 'value': ['Dress - Pleat']}","{'key': 'Inseam', 'value': ['32']}","{'key': 'Country/Region of Manufacture', 'valu...","{'key': 'Condition', 'value': ['New with tags']}",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,$125 Nwt Nautica Travelwear Wool Blend Black P...,19.99,FALSE
9,"{'key': 'Style', 'value': ['Vest']}","{'key': 'Pattern', 'value': ['Solid']}","{'key': 'Condition', 'value': ['New with tags']}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,$125 Polo Ralph Lauren Men Vest With Hood Smal...,48.50,FALSE


In [22]:
type(new_df.loc[0,0])

dict

In [23]:
features_and_counts = {}
# list of keys:
for i in range(len(df)):
    for j in range(30):
        if type(new_df.iloc[i,j])== dict:
            key = new_df.iloc[i,j]["key"]
            value = new_df.iloc[i,j]["value"]
            if key not in list(features_and_counts.keys()):
                features_and_counts.update({key:1})
            else:
                features_and_counts[key] += 1
features_and_counts

{'Sport': 764,
 'Style': 2680,
 'Condition': 2458,
 'Material': 8488,
 'Heel Height': 4193,
 'Pattern': 637,
 'Occasion': 1763,
 'Country/Region of Manufacture': 759,
 'Inseam': 261,
 'Front Style': 11,
 'Country of Manufacture': 36,
 'Color': 10363,
 'Wash': 98,
 'Gender': 11394,
 'Shoe Size': 5468,
 'Size': 4831,
 'Shoe Category': 5694,
 'Fabric Material': 1184,
 'Brand': 11243,
 'Country//Region of Manufacture': 22,
 'MPN': 92,
 'Age Group': 7139,
 'Shipping Weight (in pounds)': 834,
 'Casual & Dress Shoe Style': 4170,
 'manufacturer_part_number': 532,
 'Assembled Product Dimensions (L x W x H)': 1705,
 'Model': 6034,
 'EAN': 166,
 'Colour': 36,
 'Shoe Width': 5061,
 'Is Waterproof': 141,
 'Fabric Content': 1176,
 'Shoe Closure': 1138,
 'Manufacturer Part Number': 8121,
 'Features': 215,
 'Water-Resistant': 36,
 'Crown': 9,
 'Display': 106,
 'Case Diameter': 48,
 'Movement': 67,
 'Band Material': 73,
 'Place of Origin': 6,
 'Case Thickness': 30,
 'Number of Stones': 1,
 'Dial Color'

In [24]:
import operator
sorted_features_and_counts = sorted(features_and_counts.items(), key=operator.itemgetter(1),reverse=True)
sorted_features_and_counts

[('Gender', 11394),
 ('Brand', 11243),
 ('Color', 10363),
 ('Material', 8488),
 ('Manufacturer Part Number', 8121),
 ('Age Group', 7139),
 ('Model', 6034),
 ('Shoe Category', 5694),
 ('Shoe Size', 5468),
 ('Shoe Width', 5061),
 ('Size', 4831),
 ('Heel Height', 4193),
 ('Casual & Dress Shoe Style', 4170),
 ('Style', 2680),
 ('Condition', 2458),
 ('Occasion', 1763),
 ('Assembled Product Dimensions (L x W x H)', 1705),
 ('Fabric Material', 1184),
 ('Fabric Content', 1176),
 ('Shoe Closure', 1138),
 ('Shipping Weight (in pounds)', 834),
 ('Sport', 764),
 ('Country/Region of Manufacture', 759),
 ('Season', 677),
 ('Pattern', 637),
 ('manufacturer_part_number', 532),
 ('Type', 463),
 ('Country of Origin - Components', 439),
 ('Country of Origin - Assembly', 413),
 ('Upper Material', 287),
 ('Clothing Size', 265),
 ('Inseam', 261),
 ('Origin of Components', 242),
 ('Assembled in Country of Origin', 229),
 ('Assembled Product Weight', 228),
 ('Model No.', 227),
 ('Product in Inches (L x W x H)

In [25]:
selected_features = ['Material',
'Shoe Size',
'Color',
"Brand",
'Gender']
selected_features

['Material', 'Shoe Size', 'Color', 'Brand', 'Gender']

In [26]:
for feature in selected_features:
    for i in range(len(df)):
        for j in range(49):
            if type(new_df.iloc[i,j])== dict:
                key = new_df.iloc[i,j]["key"]
                value = new_df.iloc[i,j]["value"]
                if key == feature:
                    try:
                        new_df.at[i, feature] = value
                    except:
                        pass
    print(feature + " added...")
new_df.head()

Material added...
Shoe Size added...
Color added...
Brand added...
Gender added...


,0,1,2,3,4,5,6,7,8,9,...,47,48,name,prices.amountMin,prices.isSale,Material,Shoe Size,Color,Brand,Gender
0,"{'key': 'Sport', 'value': ['Leisure, Golf']}","{'key': 'Style', 'value': ['Pants']}","{'key': 'Condition', 'value': ['New with tags']}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,$125 Brand New - Nike Tiger Woods Tw Adaptive ...,65.00,FALSE,NaN,NaN,NaN,NaN,NaN
1,"{'key': 'Style', 'value': ['Nursing & Uniform']}","{'key': 'Material', 'value': ['Leather']}","{'key': 'Condition', 'value': ['Pre-owned']}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,$125 Dansko Mens Professional Box Leather Clog...,58.31,FALSE,Leather,NaN,NaN,NaN,NaN
2,"{'key': 'Heel Height', 'value': ['Med (1 3/4 i...","{'key': 'Pattern', 'value': ['Solid']}","{'key': 'Occasion', 'value': ['Wear to Work']}","{'key': 'Material', 'value': ['Leather']}","{'key': 'Country/Region of Manufacture', 'valu...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,$125 Dansko Professional Leather Clogs In Hick...,55.25,TRUE,[Leather],NaN,NaN,NaN,NaN
3,"{'key': 'Style', 'value': ['Golf - Dress - Ple...","{'key': 'Inseam', 'value': ['30']}","{'key': 'Material', 'value': ['65% Polyester -...","{'key': 'Front Style', 'value': ['Pleated fron...","{'key': 'Country/Region of Manufacture', 'valu...","{'key': 'Condition', 'value': ['New with tags']}",NaN,NaN,NaN,NaN,...,NaN,NaN,$125 Lauren Ralph Lauren Fully Lined Golf - Dr...,49.99,FALSE,[65% Polyester - 35% Rayon],NaN,NaN,NaN,NaN
4,"{'key': 'Style', 'value': ['Dress - Pleat']}","{'key': 'Material', 'value': ['50% Wool - 50% ...","{'key': 'Condition', 'value': ['New with tags']}","{'key': 'Country of Manufacture', 'value': ['I...","{'key': 'Color', 'value': ['Gray']}",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,$125 Lauren Ralph Lauren Lined Wool/poly Dress...,59.99,FALSE,[50% Wool - 50% Polyester],NaN,Gray,NaN,NaN


In [56]:
new_df

,0,1,2,3,4,5,6,7,8,9,...,47,48,name,prices.amountMin,prices.isSale,Material,Shoe Size,Color,Brand,Gender
0,"{'key': 'Sport', 'value': ['Leisure, Golf']}","{'key': 'Style', 'value': ['Pants']}","{'key': 'Condition', 'value': ['New with tags']}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,$125 Brand New - Nike Tiger Woods Tw Adaptive ...,65.00,FALSE,NaN,NaN,NaN,NaN,NaN
1,"{'key': 'Style', 'value': ['Nursing & Uniform']}","{'key': 'Material', 'value': ['Leather']}","{'key': 'Condition', 'value': ['Pre-owned']}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,$125 Dansko Mens Professional Box Leather Clog...,58.31,FALSE,Leather,NaN,NaN,NaN,NaN
2,"{'key': 'Heel Height', 'value': ['Med (1 3/4 i...","{'key': 'Pattern', 'value': ['Solid']}","{'key': 'Occasion', 'value': ['Wear to Work']}","{'key': 'Material', 'value': ['Leather']}","{'key': 'Country/Region of Manufacture', 'valu...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,$125 Dansko Professional Leather Clogs In Hick...,55.25,TRUE,[Leather],NaN,NaN,NaN,NaN
3,"{'key': 'Style', 'value': ['Golf - Dress - Ple...","{'key': 'Inseam', 'value': ['30']}","{'key': 'Material', 'value': ['65% Polyester -...","{'key': 'Front Style', 'value': ['Pleated fron...","{'key': 'Country/Region of Manufacture', 'valu...","{'key': 'Condition', 'value': ['New with tags']}",NaN,NaN,NaN,NaN,...,NaN,NaN,$125 Lauren Ralph Lauren Fully Lined Golf - Dr...,49.99,FALSE,[65% Polyester - 35% Rayon],NaN,NaN,NaN,NaN
4,"{'key': 'Style', 'value': ['Dress - Pleat']}","{'key': 'Material', 'value': ['50% Wool - 50% ...","{'key': 'Condition', 'value': ['New with tags']}","{'key': 'Country of Manufacture', 'value': ['I...","{'key': 'Color', 'value': ['Gray']}",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,$125 Lauren Ralph Lauren Lined Wool/poly Dress...,59.99,FALSE,[50% Wool - 50% Polyester],NaN,Gray,NaN,NaN
5,"{'key': 'Style', 'value': ['Fashion Sneakers']}","{'key': 'Condition', 'value': ['New with box']}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,$125 New Nike Women's Sz 6.5 Dunk Sky Hi Casua...,73.95,FALSE,NaN,NaN,NaN,NaN,NaN
6,"{'key': 'Style', 'value': ['Dress - Flat Front']}","{'key': 'Inseam', 'value': ['30']}","{'key': 'Country/Region of Manufacture', 'valu...","{'key': 'Condition', 'value': ['New with tags']}",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,$125 Nwt Nautica Gray Flat Front Dress Pants T...,19.99,FALSE,NaN,NaN,NaN,NaN,NaN
7,"{'key': 'Style', 'value': ['Dress - Pleat']}","{'key': 'Inseam', 'value': ['30']}","{'key': 'Country/Region of Manufacture', 'valu...","{'key': 'Condition', 'value': ['New with tags']}",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,$125 Nwt Nautica Travel Wool Blend Blue Checke...,24.99,FALSE,NaN,NaN,NaN,NaN,NaN
8,"{'key': 'Style', 'value': ['Dress - Pleat']}","{'key': 'Inseam', 'value': ['32']}","{'key': 'Country/Region of Manufacture', 'valu...","{'key': 'Condition', 'value': ['New with tags']}",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,$125 Nwt Nautica Travelwear Wool Blend Black P...,19.99,FALSE,NaN,NaN,NaN,NaN,NaN
9,"{'key': 'Style', 'value': ['Vest']}","{'key': 'Pattern', 'value': ['Solid']}","{'key': 'Condition', 'value': ['New with tags']}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,$125 Polo Ralph Lauren Men Vest With Hood Smal...,48.50,FALSE,NaN,NaN,NaN,NaN,NaN


In [144]:
final_df = new_df[['Material',
'Shoe Size',
'Color',
"Brand",
"name",
"prices.amountMin",
"prices.isSale",
'Gender'
]].dropna()
final_df = final_df.reset_index(drop=True)
final_df.head()

,Material,Shoe Size,Color,Brand,name,prices.amountMin,prices.isSale,Gender
0,[Leather],10 US,[White],Diesel,$250 Diesel Forever White Heels Pumps Shoes 10...,99.95,TRUE,Women
1,[Synthetic],[5],[BlackPatent],[Summitfashions],3 Inch Women's Sexy Gogo Boots Knee High Singl...,53.99,FALSE,[Women]
2,[polyurethane],[Shoe Size 7-10],[Gray],[Kushyfoot],3 Pairs Women���s Reusable Kushyfoot Flats To ...,7.99,FALSE,[Women]
3,"[Cloth ,�� Leather]",[M],[Black],[3N2],"3n2 7735-0101-105 Mid Cut Turf Trainer Shoe, B...",54.59,TRUE,[Men]
4,"[Cloth ,�� Leather]","[11 ,�� 12 ,�� Size 7.5 ,�� 13 ,�� 14 ,�� Size...",[Black],[3N2],"3n2 7735-0101-80 Mid Cut Turf Trainer Shoe, Bl...",53.47,TRUE,[Men]


In [145]:
# get the first element from the list in each cell 
for i in range(len(final_df)):
    for j in range(8):
        if type(final_df.iloc[i,j])== list:
            final_df.iloc[i,j] = final_df.iloc[i,j][0]
final_df = final_df.sort_values("name").reset_index(drop=True)
final_df.head()

,Material,Shoe Size,Color,Brand,name,prices.amountMin,prices.isSale,Gender
0,Leather,10 US,White,Diesel,$250 Diesel Forever White Heels Pumps Shoes 10...,99.95,TRUE,Women
1,polyurethane,Shoe Size 7-10,Gray,Kushyfoot,10-pack Men's Premium White Cabretta Golf Glov...,58.68,FALSE,Women
2,"Cloth ,�� Leather",M,Black,3N2,15051-rg-01 Montana Multi-function Black Gen. ...,545.00,FALSE,Men
3,Synthetic,9,Clear,SummitFashions,180s Womens Exolite Sonic Sport Headphone Ear ...,45.00,FALSE,Women
4,Leather,7.5,Brown,7 For All Mankind,2 Pack Barely There Berlei Contour Bras With U...,69.99,FALSE,Men


In [146]:
# clean shoe size column:
replace_words = ["Size ", "Shoe Size ", " US", "M"]
for word in replace_words:
    final_df["Shoe Size"] = final_df["Shoe Size"].str.replace(word, "")

In [147]:
# extract number only from shoe size
import re
for i in range(len(final_df)):
    final_df["Shoe Size"] = final_df["Shoe Size"].str.extract('(\d+)')
final_df

,Material,Shoe Size,Color,Brand,name,prices.amountMin,prices.isSale,Gender
0,Leather,10,White,Diesel,$250 Diesel Forever White Heels Pumps Shoes 10...,99.95,TRUE,Women
1,polyurethane,7,Gray,Kushyfoot,10-pack Men's Premium White Cabretta Golf Glov...,58.68,FALSE,Women
2,"Cloth ,�� Leather",NaN,Black,3N2,15051-rg-01 Montana Multi-function Black Gen. ...,545.00,FALSE,Men
3,Synthetic,9,Clear,SummitFashions,180s Womens Exolite Sonic Sport Headphone Ear ...,45.00,FALSE,Women
4,Leather,7,Brown,7 For All Mankind,2 Pack Barely There Berlei Contour Bras With U...,69.99,FALSE,Men
5,Synthetic,7,Brown,a.x.n.y.,2015 New Women's Fashion Flower Long Style Wra...,2.44,FALSE,Women
6,Synthetic,5,BlackPatent,Summitfashions,3 Inch Women's Sexy Gogo Boots Knee High Singl...,53.99,FALSE,Women
7,polyurethane,7,Gray,Kushyfoot,3 Pairs Women���s Reusable Kushyfoot Flats To ...,7.99,FALSE,Women
8,Leather,9,Brown,Acorn,35634012 - Suede Mid Classic+,75.00,TRUE,Women
9,Leather,13,Black,Adan,3n2 5945-0106-70 Womens Rally Metal Pt Fastpit...,94.05,FALSE,Men


In [148]:
len(final_df)

3116

In [149]:
# delete messy symbol lines:
final_df.drop(df.index[[2,7,10,11]], inplace=True)

In [ ]:
df["prices.isSale"] = df["prices.isSale"].map({"true":"TRUE", "false":"FALSE", "TRUE":"TRUE", "FALSE":"FALSE"})

# Convert Europe shoe size to US
final_df["Shoe Size"] = final_df["Shoe Size"].map()


In [150]:
# save to csv
final_df.to_csv(path_or_buf="clean_data.csv", index=False)

In [151]:
df = pd.read_csv("clean_data.csv")
df

,Material,Shoe Size,Color,Brand,name,prices.amountMin,prices.isSale,Gender
0,Leather,10.0,White,Diesel,$250 Diesel Forever White Heels Pumps Shoes 10...,99.95,True,Women
1,polyurethane,7.0,Gray,Kushyfoot,10-pack Men's Premium White Cabretta Golf Glov...,58.68,False,Women
2,Synthetic,9.0,Clear,SummitFashions,180s Womens Exolite Sonic Sport Headphone Ear ...,45.00,False,Women
3,Leather,7.0,Brown,7 For All Mankind,2 Pack Barely There Berlei Contour Bras With U...,69.99,False,Men
4,Synthetic,7.0,Brown,a.x.n.y.,2015 New Women's Fashion Flower Long Style Wra...,2.44,False,Women
5,Synthetic,5.0,BlackPatent,Summitfashions,3 Inch Women's Sexy Gogo Boots Knee High Singl...,53.99,False,Women
6,Leather,9.0,Brown,Acorn,35634012 - Suede Mid Classic+,75.00,True,Women
7,Leather,13.0,Black,Adan,3n2 5945-0106-70 Womens Rally Metal Pt Fastpit...,94.05,False,Men
8,Synthetic,9.0,Clear,SummitFashions,4 Inch Sexy High Heel Mule Slide Shoe With Rhi...,64.99,False,Women
9,Glitter Polyurethane,6.0,Green Glitter,ELLIE SHOES,414patty Heel Adult,57.56,False,Unisex


In [152]:
len(df)

3112

In [172]:
df["Material"].value_counts()[:4]

Leather      915
Synthetic    871
Canvas       366
Suede        307
Name: Material, dtype: int64

In [167]:
df["Shoe Size"].value_counts()

8.0     545
7.0     543
6.0     473
9.0     464
10.0    323
5.0     218
11.0    212
12.0     94
13.0     58
14.0     21
4.0      19
2.0      11
15.0      8
38.0      7
37.0      6
39.0      5
44.0      5
3.0       5
41.0      5
42.0      3
40.0      3
0.0       3
36.0      2
1.0       1
17.0      1
16.0      1
45.0      1
Name: Shoe Size, dtype: int64

In [171]:
df["Color"].value_counts()

Black                                       834
Brown                                       388
Beige                                       205
Gray                                        175
Blue                                        173
White                                       156
Silver                                      125
Red                                         112
Green                                       101
Multicolor                                   94
Gold                                         68
Pink                                         57
Orange                                       48
Yellow                                       46
Other                                        23
Clear                                        20
Brass                                        18
Purple                                       16
Off-White                                    15
Grey                                          9
Dark Brown                              

In [170]:
df["Brand"].value_counts()

Nike                          123
TOMS                           65
Brinley Co.                    53
Unique Bargains                52
Easy Spirit                    51
Bearpaw                        41
Allegra K                      41
PUMA                           41
Nine West                      41
VANS                           37
Marc Fisher                    35
Gameday Boots                  34
Alpine Swiss                   33
New Balance                    31
Calvin Klein                   31
Alfani                         31
Fitflop                        29
Bamboo                         28
Jordan                         27
Me Too                         27
Propet                         25
SummitFashions                 25
Style & Co.                    24
MICHAEL Michael Kors           22
MUK LUKS                       21
Dearfoams                      21
Reebok                         21
BareTraps                      19
Kenneth Cole Reaction          19
Kenneth Cole N

In [169]:
df["Gender"].value_counts()

Women     2068
Men        942
Unisex      72
Girls       17
Boys        13
Name: Gender, dtype: int64